# HERCULES-EXTRACTION

HERCULES-EXTRACTION est un outil d'extraction qui a pour but l'extraction d'entitées nommées dans un texte. Pour ce faire, nous utilisons des outils et APIs déjà fonctionels pour les textes en anglais. Pour être capable d'utiliser ces outils et APIs, nous devons tout d'abord [traduire le texte](#Traduction-du-texte) en anglais. Après avoir avoir traduit le texte, nous utilisons les différents outils et APIs pour [extraire les entitées nommées](#Extraction-d'entitées). Nous utilisons aussi une approche de [résolution de la coréférence](#Résolution-de-la-coréférence) afin de filtrer les entitéess précédement extraites. Enfin, nous [traduisons les entitées](#Traduction-des-entitées) extraites en français et nous [exportons](#Exportation) les triplets sous format rdf.

## Configuration

In [ ]:
import os
from pathlib import Path
import requests
import subprocess
import sys
import zipfile

Configurons le notebook.

In [ ]:
setup_path = Path('setup')

**Prérequis**:
- Java 8

Installons les requis.

In [ ]:
!{sys.executable} -m pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

Configuront les composantes.

**AzureTranslator**

Définissons la variable d'environnement `AZURE_TOKEN` à notre clé d'Azure Text API. 

In [ ]:
os.environ['AZURE_TOKEN'] = ''

**GoogleCloudTranslator** and **GoogleEntityExtractor**

Définissons la variable d'environnement `GOOGLE_APPLICATION_CREDENTIALS` à notre « Google service account JSON keyfile ».

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''

**MyMemoryTranslator**

Définissons la variable d'environnement `MYMEMORY_TOKEN` à notre clé de MyMemory API.

In [ ]:
os.environ['MYMEMORY_TOKEN'] = ''

**DandelionEntityExtractor**

Définissons la variable d'environnement `DANDELION_TOKEN` à notre clé de Dandelion API.

In [ ]:
os.environ['DANDELION_TOKEN'] = ''

**StanfordCoreferenceResolver**

Téléchargeons le serveur Stanford CoreNLP.

In [ ]:
corenlp_zip_path = setup_path / 'stanford-corenlp-full-2018-10-05.zip'
corenlp_dir_path = setup_path / 'stanford-corenlp-full-2018-10-05'
corenlp_url = 'http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip'

setup_path.mkdir(parents=True, exist_ok=True) 

if not corenlp_zip_path.is_file():
    response = requests.get(corenlp_url)
    with corenlp_zip_path.open('wb') as f:
        f.write(response.content)

if not corenlp_dir_path.is_dir():
    with zipfile.ZipFile(corenlp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(setup_path)

Démarrons le serveur Stanford CoreNLP.

In [ ]:
corenlp_server = subprocess.Popen(['java', '-Xmx5G', '-cp', str(corenlp_dir_path.resolve() / '*'), 'edu.stanford.nlp.pipeline.StanfordCoreNLPServer', '-port 9000', '-timeout 60000', '-threads 5', '-maxCharLength 100000', '-quiet True', '-preload tokenize,ssplit,pos,lemma,ner,parse,coref'])

Définissons la variable d'environnement `CORENLP_HOME` au chemin du serveur Stanford CoreNLP.

In [ ]:
os.environ['CORENLP_HOME'] = str(corenlp_dir_path.resolve())

## Valeurs initiales

In [ ]:
import translation
import extraction
import coreference
import export

Allons chercher le texte à partir d'un fichier. Pour cet exemple, nous avons choisis un text à propos de la Basilique Notre-Dame.

In [ ]:
text_language = 'fr'
extraction_language = 'en'

text_path = Path('sample', 'default', 'text.txt')
text = text_path.read_text(encoding='utf-8')
print(text)

## Traduction du texte

Traduisons le texte en anglais.

**AzureTranslator**

Ce traducteur utilise l'API d'Azure Text.

In [ ]:
azure_translator = translation.AzureTranslator()
azure_translated_text = azure_translator.translate(text, text_language, extraction_language)
print(azure_translated_text)

**GoogleCloudTranslator**

Ce traducteur utilise l'API de Google Translation Cloud.

In [ ]:
google_translator = translation.GoogleCloudTranslator()
google_translated_text = google_translator.translate(text, text_language, extraction_language)
print(google_translated_text)

**GoogletransTranslator**

Ce traducteur utilise le site web de Google Translation.

In [ ]:
googletrans_translator = translation.GoogletransTranslator()
googletrans_translated_text = googletrans_translator.translate(text, text_language, extraction_language)

print(googletrans_translated_text)

**MyMemoryTranslator**

Ce traducteur utilise l'API de MyMemory.

In [ ]:
my_memory_translator = translation.MyMemoryTranslator()
my_memory_translated_text = my_memory_translator.translate(text, text_language, extraction_language)
print(my_memory_translated_text)

Prenons le texte traduit par MyMemory pour les prochaines étapes.

In [ ]:
translator = my_memory_translator
translated_text = my_memory_translated_text

## Extraction d'entitées

Extractons les entitées à partir du texte traduit.

**DandelionEntityExtractor**

Cet extracteur d'entitées utilise l'API de Dandelion.

In [ ]:
dandelion_entity_extractor = extraction.DandelionEntityExtractor()
dandelion_entities = dandelion_entity_extractor.extract_entities(translated_text)
for entity in dandelion_entities:
    print(entity)

**GoogleEntityExtractor**

Cet extracteur d'entitées utilise l'API de Google Natural Language.

In [ ]:
google_entity_extractor = extraction.GoogleEntityExtractor()
google_entities = google_entity_extractor.extract_entities(translated_text)
for entity in google_entities:
    print(entity)

Prenons les entitées extraites par Dandelion pour les prochaines étapes.

In [ ]:
extracted_entities = dandelion_entities

## Résolution de la coréférence

In [ ]:
skip_coreference = False

Filtrons les entitées précédement extraites en utilisant de la résolution de coréférence.

In [ ]:
def get_relevant_entity_from_mention(mention):
    if len(mention) <= 0:
        return None
    for entity in mention:
        if entity.entity_type != extraction.EntityType.THING:
            return entity
    return mention[0]

**StanfordCoreferenceResolver**

Ce résolveur de coréférence utilise une instance locale du serveur de Stanford CoreNLP. S'il y a une erreur `Read timed out`, vous pouvez passer cette étape en changeant `skip_coreference` à `True`.

In [ ]:
if not skip_coreference:
    stanford_coreference_resolver = coreference.StanfordCoreferenceResolver(start_server=False, endpoint='http://localhost:9000')
    stanford_mentions = stanford_coreference_resolver.resolve_coreferences(translated_text, extracted_entities)

    stanford_filtered_entities = []
    for mention in stanford_mentions:
        entity = get_relevant_entity_from_mention(mention)
        if entity is not None:
            stanford_filtered_entities.append(entity)

    for entity in stanford_filtered_entities:
        print(entity)

Prenons les entitées filtrées avec Stanford pour les prochaines étapes.

In [ ]:
if skip_coreference:
    filtered_entities = extracted_entities
else:
    filtered_entities = stanford_filtered_entities

## Traduction des entitées

Traduisons les entitées filtrées en français.

In [ ]:
translated_back_entities = []
for entity in filtered_entities:
    entity_name = translator.translate(entity.name, extraction_language, text_language)
    translated_back_entity = extraction.Entity(entity_name, entity.entity_type, None, None)
    translated_back_entities.append(translated_back_entity)
    print(translated_back_entity)

## Exportation

Exportons les entitées traduites vers un fichier rdf.

In [ ]:
export_path = Path('notebook-export')
export_path.mkdir(parents=True, exist_ok=True) 

**CIDOCCRMExporter**

Cet exporteur est conçu pour fonctionner avec l'ontologie de la [CIDOC CRM](http://www.cidoc-crm.org/).

In [ ]:
export_language = 'turtle'
entity_namespace = 'http://culture.gouv.qc.ca/entity/'
ontology_namespace = 'http://www.cidoc-crm.org/cidoc-crm/'
cidoccrm_export_path = export_path / 'cidoccrm.ttl'

cidoccrm_exporter = export.CIDOCCRMExporter()
cidoccrm_export = cidoccrm_exporter.export(translated_back_entities, entity_namespace, ontology_namespace, export_language)

cidoccrm_export_path.write_text(cidoccrm_export, encoding='utf-8')
print(cidoccrm_export)

## Nettoyage du notebook

Arrête le serveur CoreNLP créé par ce notebook.

In [ ]:
corenlp_server.kill()